In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0


In [4]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re

# Authenticate and create the service object
auth.authenticate_user()
service = build('docs', 'v1')
drive_service = build('drive', 'v3')

# Markdown content
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


def parse_markdown(markdown):
    lines = markdown.split('\n')
    parsed_content = []
    current_heading_level = 0

    for line in lines:
        line = line.strip()
        if line.startswith('#'):
            heading_level = line.count('#')
            text = line.strip('#').strip()
            parsed_content.append({'type': 'heading', 'level': heading_level, 'text': text})
            current_heading_level = heading_level
        elif line.startswith('- [ ]'):
            parsed_content.append({'type': 'checkbox', 'text': line[4:].strip()})
        elif line.startswith('-'):
            parsed_content.append({'type': 'bullet', 'level': current_heading_level + 1, 'text': line[1:].strip()})
        elif line:
            parsed_content.append({'type': 'text', 'text': line})
        else:
            parsed_content.append({'type': 'newline'})

    return parsed_content


def create_google_doc(service, title):
    try:
        body = {'title': title}
        doc = service.documents().create(body=body).execute()
        return doc['documentId']
    except HttpError as error:
        print(f"An error occurred while creating the document: {error}")
        return None


def format_doc(service, document_id, parsed_content):
    requests = []

    def add_text(text, index, style=None):
        requests.append({
            'insertText': {
                'location': {'index': index},
                'text': text
            }
        })
        if style:
            requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': index, 'endIndex': index + len(text)},
                    'textStyle': style,
                    'fields': 'bold'
                }
            })

    def add_heading(text, index, level):
        add_text(text + '\n', index, {
            'bold': True,
            'fontSize': {'magnitude': 11 + (3 - level) * 2, 'unit': 'PT'}
        })

    def add_bullet(text, index, level):
        add_text(text + '\n', index, {
            'bold': False,
            'fontSize': {'magnitude': 11, 'unit': 'PT'}
        })

    def add_checkbox(text, index):
        add_text(text + '\n', index, {
            'bold': True,
            'fontSize': {'magnitude': 11, 'unit': 'PT'}
        })

    index = 1
    for item in parsed_content:
        if item['type'] == 'heading':
            add_heading(item['text'], index, item['level'])
            index += len(item['text']) + 1
        elif item['type'] == 'bullet':
            add_bullet(item['text'], index, item['level'])
            index += len(item['text']) + 1
        elif item['type'] == 'checkbox':
            add_checkbox(item['text'], index)
            index += len(item['text']) + 1
        elif item['type'] == 'text':
            add_text(item['text'] + '\n', index)
            index += len(item['text']) + 1
        elif item['type'] == 'newline':
            add_text('\n', index)
            index += 1

    try:
        body = {'requests': requests}
        service.documents().batchUpdate(documentId=document_id, body=body).execute()
    except HttpError as error:
        print(f"An error occurred while formatting the document: {error}")


def share_document(service, document_id):
    try:
        permission = {'type': 'anyone', 'role': 'reader'}
        service.permissions().create(fileId=document_id, body=permission).execute()
    except HttpError as error:
        print(f"An error occurred while sharing the document: {error}")


def main():
    parsed_content = parse_markdown(markdown_content)
    document_id = create_google_doc(service, "Product Team Sync - May 15, 2023")
    if document_id:
        format_doc(service, document_id, parsed_content)
        share_document(drive_service, document_id)
        doc_url = f"https://docs.google.com/document/d/{document_id}/edit"
        print(f"Document created and shared: {doc_url}")


if __name__ == "__main__":
    main()

Document created and shared: https://docs.google.com/document/d/1LJAEhCtT9IJtF7Qml-PKvKubPDX2bGnFg0lYb9KQSOM/edit
